## Load Packages

In [ ]:
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

from sklearn.preprocessing import PowerTransformer, LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer,  TransformedTargetRegressor

from env import *
from plotting.plot import *
from processing.data import Data

pd.pandas.set_option('display.max_columns', None)

# Import Data

In [ ]:

train = Data.from_csv(filepath=DIR_DATA_TRAIN, index_col=COL_ID)
test  = Data.from_csv(filepath=DIR_DATA_TEST, index_col=COL_ID)
print(train)
repr(train)

In [ ]:
# Create more human-readable columns
for regex, repl in REGEX_REPL_COLUMN:
    train.columns = train.columns.map(lambda x: re.sub(string=x, pattern=regex, repl=repl))
    test.columns = test.columns.map(lambda x: re.sub(string=x, pattern=regex, repl=repl))

train.columns = train.columns.map(str.lower)
test.columns  = test.columns.map(str.lower)

train.col_target = COL_TARGET
test.col_target = COL_TARGET

print(f'Columns of training data: {train.columns.tolist()}')

***
# Exploratory Data Analysis

## Variable Types

In [ ]:
train.print_column_types()

In [ ]:
train.df.info()

***
# Process Data

## Remove Uninformative Features

In [ ]:
train.drop_columns_regex(regex='^misc')

## Unique Values

In [ ]:
plot_count_unique(train.df, 
                  dtype_include=['float64', 'int64'],
                  title='Count of unique values - numerical fields')

In [ ]:
plot_count_unique(train.df, 
                  dtype_include=['O'],
                  title='Count of unique values - categorical fields')

In [ ]:
train.remove_constant_columns()

### Aggregate low count categorical features to 'other' category

In [ ]:
cat_count_unique = train.count_unique(dtypes='object')

CATEGORICAL_COL_COUNT_THRESHOLD = 10
CATEGORICAL_GROUPING_THRESHOLD = 0.01

for col in cat_count_unique.index[cat_count_unique > CATEGORICAL_COL_COUNT_THRESHOLD]:

    # Apply grouping of categories to training set
    cat_group_mapping = train.group_low_count_categories(col=col, fill_value='Other', threshold=CATEGORICAL_GROUPING_THRESHOLD)
    
    # Apply grouping of categories to test set
    test.df[col] = test.df[col].replace(cat_group_mapping)

## Missing Values

In [ ]:
col_missing_bool = train.df.apply(lambda x: x.isnull().sum() > 0)
col_missing_desc = train.df.columns[col_missing_bool].tolist()

print(f'Columns with missing values:')
print("\n".join(col_missing_desc))

In [ ]:
sns.set_theme(
    style="ticks", 
    # palette=sns.color_palette("Set1"), 
    rc=custom_params)

# Plot percentage of missing data
for dtypes in [['object',], ['int64', 'float64']]:

    plot_perc_missing(
        train.df, 
        title=f'% of Missing Values - {dtypes} Columns',
        dtype_include=dtypes,
        )

### Remove columns with many missing values

In [ ]:
train.remove_missing_columns(threshold=MISSING_THRESHOLD_DROP)

### Create flag for missing values

In [ ]:
col_missing_bool = train.df.apply(lambda x: x.isnull().sum() > 0)
col_missing_desc = train.df.columns[col_missing_bool].tolist()

for col in col_missing_desc:

    boxplot_histogram_missingness_relationship(
        df=train.df, 
        col_var=col, 
        col_target=train.col_target, 
        add_kde=True,
        add_yeojohnson=True)

In [ ]:
train.add_flag_missing_values(
    ttest_threshold=FLAG_MISSING_PVALUE_TRESHOLD,
    ttest_min_samples=FLAG_MISSING_MIN_SAMPLES, 
    yeojohnson_transform=True
    )


## Datetime Features

### Create features from time data

In [ ]:
train.create_column(
    col_name='year_built_or_remod', 
    values=train.df[['year_remod_add', 'year_built']].apply(np.max, axis=1))

def x(a,b):
    return np.max([a-b, 0])

# Calculate age when sold
train.create_column(
    col_name='age_garage_when_sold',
    values=train.df.apply(lambda f: x(f['year_sold'], f['garage_year_built']), axis=1))

train.create_column(
    col_name='age_house_when_sold', 
    values=train.df.apply(lambda f: x(f['year_sold'], f['year_built_or_remod']), axis=1))

train.drop_columns_regex(regex='year(?!.+missing$)')

## Discrete Features

In [ ]:
# ms_sub_class is a categorical feature
train.change_column_types({'ms_sub_class': 'object'})

In [ ]:
train.print_column_types()

In [ ]:
discrete_cols   = [col for col in train.num_columns if len(train.df[col].unique()) < 20]
continuous_cols = [col for col in train.num_columns if col not in discrete_cols]

train.df[discrete_cols].head(10)

### Relationship with Target Feature

In [ ]:
for col in discrete_cols:
    
    sns.set_theme(
            style="ticks", 
            palette=sns.color_palette("Reds"), 
            rc=custom_params)
    
    sns.catplot(data=train.df, x=col, y=COL_TARGET, kind='box', height=4.5, aspect=1.7)
    sns.stripplot(data=train.df, x=col, y=COL_TARGET, jitter=0.1, alpha=0.1, color='k')
    plt.title(f'Distribution of \'{COL_TARGET}\' by \'{col}\'')
    plt.ylabel(COL_TARGET.replace('_', ' ').upper())
    plt.xlabel(col.replace('_', ' ').upper())
    plt.show()

## Numerical features

In [ ]:
train.df[continuous_cols].head()

In [ ]:
sns.set_theme(
            style="white", 
            palette=sns.color_palette("Set2"), 
            rc={
                "axes.grid": False, 
                "grid.color": '0.95', 
                "axes.spines.right": False, 
                "axes.spines.top": False
                })

train.df.loc[:, continuous_cols].hist(bins=15, figsize=(15,15))
plt.suptitle(f'Distribution of Values for Numerical Features')
plt.show()

### Analyse Feature Correlation

In [ ]:
plot_correlation_heatmap(train.df)

### Remove Features with low correlation to target

In [ ]:
plot_feature_correlations(train.df, col_target=train.col_target, threshold=CORRELATION_TARGET_THRESHOLD)

In [ ]:
train.remove_correlation_features(threshold=CORRELATION_TARGET_THRESHOLD)

### Process numerical features with high collinearity

In [ ]:
train.remove_highly_collinear_features(threshold=COLLINEARITY_THRESHOLD)

### Process highly correlated categorical features

In [ ]:
CRAMER_MAX_CARDINALITY = 4
CATEGORICAL_CORRELATION_THRESHOLD = 0.80
CATEGORICAL_CORRELATION_SELECTION_STRATEGY = 'cardinality' # 'cardinality' 

feats = train.remove_highly_correlated_categorical_features(
    cramer_max_cardinality=CRAMER_MAX_CARDINALITY,
    cramer_threshold=CATEGORICAL_CORRELATION_THRESHOLD,
    selection_strategy=CATEGORICAL_CORRELATION_SELECTION_STRATEGY
)